In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

In [ ]:
!pip install feedparser
!pip install pdfplumber
!pip install pypdf2
import os
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import csv
import PyPDF2 as pdf 
import pdfplumber



In [ ]:
!pip install transformers

In [ ]:
#importation des rapports
def import_text(ch):
 textepdf=""  
 f = open(ch,'rb')
 reader=pdf.PdfFileReader(f,strict = False)
 if reader.isEncrypted:
     reader.decrypt('hoge1234')
     print(f"Number of page: {reader.getNumPages()}") 
 
 num = reader.getNumPages()
 with pdfplumber.open(ch) as f:  
  page = f.pages[0]
  zz = page.extract_text()
  textepdf += str(zz)
  for i in range(1, num):
    page = f.pages[i]
    zz = page.extract_text()
    zz = str(zz)
    textepdf += zz
 return textepdf

 


In [ ]:
import re
import string
#remove ponctuation and numbers

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub(r'[0-9]','',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…€•–°º]', '', text)
    #text = re.sub('\n', '', text)
    return text
   

In [ ]:
rapport = clean_text(import_text('/content/drive/MyDrive/Solvay/2014 Annual Report_0_0.pdf'))
rapport = rapport[1920:9900]
#rapport='what we have achieved in has made us better equipped to deal with the challenges ahead'

In [ ]:
print(rapport)

of the world
 
latin america
  of net sales
 employees
 industrial sites
 major ri centers 
of net sales
 employees
 industrial sites
 major ri center
solvaycom  annual report
group profile
our results  
and investments
 
 
financial indicators
continuing operations
net sales rebitda adujsted net income  capital 
in  million  in  million solvay share expenditures
in  million in  million 

 

 


                            
 
the usbased eco services business has been disposed as of december st  and has been reported under assets held for sale 
and discontinued operations as from the third quarter of  for comparative purposes  and  income and cash flow 
statements data have been restated for eco services business discontinuation as well as for the updated reallocation of shared 
functions costs from the corporate  business services segment into the global business units 
 and  financial statements reflect the groups application of ifrs 
furthermore solvay presents adjusted income state

In [ ]:
from transformers import  BertForSequenceClassification , BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')

In [ ]:

tokens = tokenizer.encode_plus(rapport,add_special_tokens=True,max_length=512,
                               truncation = True,padding="max_length")
tokens

{'input_ids': [101, 1997, 1996, 2088, 3763, 2637, 1997, 5658, 4341, 5126, 3919, 4573, 2350, 15544, 6401, 1997, 5658, 4341, 5126, 3919, 4573, 2350, 15544, 2415, 14017, 3567, 2100, 9006, 3296, 3189, 2177, 6337, 2256, 3463, 1998, 10518, 3361, 20390, 5719, 3136, 5658, 4341, 2128, 16313, 2850, 4748, 23049, 14701, 5658, 3318, 3007, 1999, 2454, 1999, 2454, 14017, 3567, 2100, 3745, 22697, 1999, 2454, 1999, 2454, 1996, 18833, 11022, 2094, 17338, 2578, 2449, 2038, 2042, 21866, 2004, 1997, 2285, 2358, 1998, 2038, 2042, 2988, 2104, 7045, 2218, 2005, 5096, 1998, 8944, 3136, 2004, 2013, 1996, 2353, 4284, 1997, 2005, 12596, 5682, 1998, 3318, 1998, 5356, 4834, 8635, 2951, 2031, 2042, 2717, 4383, 2005, 17338, 2578, 2449, 12532, 16778, 11231, 3370, 2004, 2092, 2004, 2005, 1996, 7172, 2613, 4135, 10719, 1997, 4207, 4972, 5366, 2013, 1996, 5971, 2449, 2578, 6903, 2046, 1996, 3795, 2449, 3197, 1998, 3361, 8635, 8339, 1996, 2967, 4646, 1997, 2065, 2869, 7297, 14017, 3567, 2100, 7534, 10426, 3318, 4861, 2836

In [ ]:

tokens = tokenizer.encode_plus(rapport,add_special_tokens=False,
                               return_tensors='pt')
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (1288 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[1997, 1996, 2088,  ..., 3737, 4013, 2094]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:

input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)
chunksize = 512
input_id_chunks=list(input_id_chunks)
mask_chunks=list(mask_chunks)

for i in range(len(input_id_chunks)):
   input_id_chunks[i]=torch.cat([
      torch.Tensor([101]),input_id_chunks[i],torch.Tensor([102])                          
   ])
   mask_chunks[i]=torch.cat([
      torch.Tensor([1]),mask_chunks[i],torch.Tensor([1])                      
   ]) 
  
   pad_len = chunksize - input_id_chunks[i].shape[0]
   if pad_len > 0:
     input_id_chunks[i] = torch.cat([
          input_id_chunks[i],torch.Tensor([0]* pad_len)                          
     ])
     mask_chunks[i] = torch.cat([
          mask_chunks[i],torch.Tensor([0]* pad_len)                          
     ])
input_ids = torch.stack(input_id_chunks)
attention_mask = torch.stack(mask_chunks)

input_dict = { 'input_ids':input_ids.long(), 'attention_mask':attention_mask.int() }
input_dict  


{'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32),
 'input_ids': tensor([[  101,  1997,  1996,  ...,  4997,  4742,   102],
         [  101,  4525,  2013,  ..., 15770,  8231,   102],
         [  101,  5301,  3791,  ...,     0,     0,     0]])}

In [ ]:
output = model(**input_dict)


In [ ]:
probs = torch.nn.functional.softmax(output[0],dim=-1)
probs

tensor([[0.0481, 0.0237, 0.9282],
        [0.1693, 0.0156, 0.8151],
        [0.5139, 0.0100, 0.4762]], grad_fn=<SoftmaxBackward>)

In [ ]:
mean = probs.mean(dim=0)
mean   

tensor([0.2437, 0.0164, 0.7398], grad_fn=<MeanBackward1>)

In [ ]:
if (mean[0]-mean[1]< 0):
  print('negative')
if (0<mean[0]-mean[1]< 0.5):
  print('positive')    
if (mean[0]-mean[1]> 0.5):
  print('positive, trés bonne situation')  
print('son score est :')
print(mean[0]-mean[1])  

positive
son score est :
tensor(0.2273, grad_fn=<SubBackward0>)
